In [127]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import bs4 as bs

import geocoder # import geocoder
print('Libraries imported.')

Libraries imported.


### Parsing the data from Wikipedia

In [128]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url)
soup = bs.BeautifulSoup(source.text, 'html.parser')

In [129]:
df = pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [130]:
df.columns = ["PostalCode", "Borough", "Neighborhood"];
# ignore borough with not assigned
df = df[~df.Borough.str.contains("Not")]

# if postal code area has borough but no neighborhood, then the neighbohood is the same as borough
df.Neighborhood = df.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)

# group neighborhoods by postal code
df_result = df.groupby(["PostalCode", "Borough"]).agg(lambda col: ', '.join(col)).sort_values("PostalCode")

# adding the index column
df_result = df_result.reset_index()

In [131]:
df_result.shape

(103, 3)

### Geolocation

In [153]:
r = pd.read_csv("Geospatial_Coordinates.csv")
toronto_geoloc = pd.merge(df_result, r, left_on="PostalCode", right_on="Postal Code").drop(labels="Postal Code", axis=1)
toronto_geoloc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
